In [6]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import TheilSenRegressor, RANSACRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor, GradientBoostingRegressor



In [7]:
df = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/train.csv")

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


test_df = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/test.csv")
submission_df = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/sample_solution.csv")

def GDR(data, target):
    """
    This function trains an ElasticNet model on the provided data and target,
    and returns the trained model. This function is intended to be used for
    generating predictions for the submission file.
    """
    X = data
    y = target

    model = make_pipeline(
        StandardScaler(),
        GaussianProcessRegressor(kernel=C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0), n_restarts_optimizer=5, random_state=42)
    )
    model.fit(X, y)
    return model

# Testing loop and submission generation
for i in range(1, 11):
  model = GDR(df[['Component1_fraction',
                  'Component2_fraction',
                  'Component3_fraction',
                  'Component4_fraction',
                  'Component5_fraction',
                  f'Component1_Property{i}',
                  f'Component2_Property{i}',
                  f'Component3_Property{i}',
                  f'Component4_Property{i}',
                  f'Component5_Property{i}']],
              df[f"BlendProperty{i}"]) # Note: Pass the target as a Series, not a DataFrame

  # Make predictions on the test data for this property
  test_predictions = model.predict(test_df[['Component1_fraction',
                                            'Component2_fraction',
                                            'Component3_fraction',
                                            'Component4_fraction',
                                            'Component5_fraction',
                                            f'Component1_Property{i}',
                                            f'Component2_Property{i}',
                                            f'Component3_Property{i}',
                                            f'Component4_Property{i}',
                                            f'Component5_Property{i}']])

  # Update the submission DataFrame
  submission_df[f'BlendProperty{i}'] = test_predictions

# Save the submission file
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge after 11 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge after 8 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge after 11 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown i

Submission file 'submission.csv' created successfully.


In [ ]:
# Select the relevant columns from test_df for prediction
feature_columns = [
    'Component1_fraction',
    'Component2_fraction',
    'Component3_fraction',
    'Component4_fraction',
    'Component5_fraction',
    'Component1_Property10',
    'Component2_Property10',
    'Component3_Property10',
    'Component4_Property10',
    'Component5_Property10'
]

# Prepare input features
X_test = test_df[feature_columns]

# Create the model pipeline
model = make_pipeline(
    StandardScaler(),
    GaussianProcessRegressor(kernel=C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0), n_restarts_optimizer=5, random_state=42)
)

# Fit the model on training data for BlendProperty10
X_train = df[feature_columns]
y_train = df['BlendProperty10']
model.fit(X_train, y_train)

# Predict on test data
output = model.predict(X_test)
print(output)

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np

def GPR():
    """Gaussian Process Regression with MAPE evaluation."""
    
    # Load data
    train = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/train.csv")
    test = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/test.csv")
    
    # Save test IDs for submission
    test_ids = test["ID"]
    test = test.drop("ID", axis=1)

    # Split into input features and target variables
    X = train.iloc[:, :55]     # First 55 columns = features
    y = train.iloc[:, 55:]     # Last 10 columns = target variables

    # Define kernel for GPR: Constant * RBF
    kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0)

    # Build pipeline with feature scaling and GPR model
    model = make_pipeline(
        StandardScaler(),
        GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, random_state=42)
    )

    # Fit model
    model.fit(X, y)

    # Predict on test data
    predictions = model.predict(test)

    # Ensure predictions are float64
    predictions = predictions.astype(np.float64)

    # Prepare submission
    prediction_columns = [f'BlendProperty{i}' for i in range(1, 11)]
    submission = pd.DataFrame(predictions, columns=prediction_columns)
    submission.insert(0, "ID", test_ids.astype(int))  # Make sure ID is int if needed

    return submission


In [39]:
x = GPR()

In [40]:
x.columns

Index(['ID', 'BlendProperty1', 'BlendProperty2', 'BlendProperty3',
       'BlendProperty4', 'BlendProperty5', 'BlendProperty6', 'BlendProperty7',
       'BlendProperty8', 'BlendProperty9', 'BlendProperty10'],
      dtype='object')

In [ ]:
x.to_csv("LOL.csv")\


In [2]:
import pandas as pd

In [3]:
l = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/heheeee.csv")

In [4]:
l.head()

,Unnamed: 0,ID,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0,1,0.076050,0.194261,0.518081,0.608311,0.748555,0.700488,0.532898,0.230367,-0.458778,0.391746
1,1,2,-0.785863,-0.752607,-1.240424,-0.079601,-0.152841,-0.285262,-1.279907,-1.143425,-1.093542,-0.008458
2,2,3,1.831939,1.176495,1.014532,1.082408,0.351342,1.623166,0.940543,2.079502,0.460189,2.129905
3,3,4,-0.350278,0.168671,0.851169,-0.422374,1.568086,-0.390580,0.822842,1.208657,0.293349,-0.909596
4,4,5,0.296417,-0.951231,1.001594,0.457759,0.516471,-0.013109,0.953690,-0.147693,-0.547510,0.971807


In [5]:
l.columns

Index(['Unnamed: 0', 'ID', 'BlendProperty1', 'BlendProperty2',
       'BlendProperty3', 'BlendProperty4', 'BlendProperty5', 'BlendProperty6',
       'BlendProperty7', 'BlendProperty8', 'BlendProperty9',
       'BlendProperty10'],
      dtype='object')

In [6]:
l = l.drop(['Unnamed: 0'], axis=1)

In [7]:
l.columns

Index(['ID', 'BlendProperty1', 'BlendProperty2', 'BlendProperty3',
       'BlendProperty4', 'BlendProperty5', 'BlendProperty6', 'BlendProperty7',
       'BlendProperty8', 'BlendProperty9', 'BlendProperty10'],
      dtype='object')

In [8]:
l.to_csv("SASA.csv")

In [37]:
# Remove rows from l where any value is not a floating point number
l = l[l.applymap(lambda x: isinstance(x, float)).all(axis=1)]
l.to_csv("hehehe.csv")

/var/folders/pj/jwdq4dd92f54s4h77g71lj8r0000gn/T/ipykernel_5255/801062793.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  l = l[l.applymap(lambda x: isinstance(x, float)).all(axis=1)]
